In [1]:
import gurobipy as gp
from gurobipy import GRB

## Parameters

In [2]:
arcs, distances = gp.multidict({
    (1, 2): 25,
    (1, 3): 20,
    #
    (2, 3):  3,
    (2, 4):  5,
    (2, 6): 14,
    #
    (3, 2):  3,
    (3, 5):  6,
    #
    (4, 2):  5,
    (4, 5):  4,
    (4, 6):  4,
    #
    (5, 3):  6,
    (5, 4):  4,
    (5, 6):  7,
})

supply = {
    1:1,
}

demand = {
    2: 0,
    3: 0,
    4: 0,
    5: 0,
    6: 1,
}

# Model
## Decision Variables

In [3]:
m = gp.Model('shortest_path')
path = m.addVars(arcs, vtype=GRB.CONTINUOUS, name='path')

Set parameter Username
Academic license - for non-commercial use only - expires 2022-11-19


## Objective Function

In [4]:
Z = path.prod(distances)
m.ModelSense = GRB.MINIMIZE
m.setObjective(Z)

## Constraints

In [5]:
source = m.addConstrs(
    (path.sum(i,'*') <= s for i,s in supply.items()),
    name='source'
)

sink = m.addConstrs(
    (path.sum('*', j) - path.sum(j, '*') == d for j,d in demand.items()),
    name='sink'
)

In [6]:
m.update()
print(f'Number of variables: {m.NumVars}')
print(f'Number of constraints: {m.NumConstrs}')
print(f'Constraints are \n {m.getConstrs()} \n')

Number of variables: 13
Number of constraints: 6
Constraints are 
 [<gurobi.Constr source[1,1]>, <gurobi.Constr sink[2,0]>, <gurobi.Constr sink[3,0]>, <gurobi.Constr sink[4,0]>, <gurobi.Constr sink[5,0]>, <gurobi.Constr sink[6,1]>] 



## Optimization process

In [7]:
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6 rows, 13 columns and 26 nonzeros
Model fingerprint: 0xdf452c01
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 6 rows, 13 columns, 26 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0000000e+00   1.000000e+00   0.000000e+00      0s
       4    3.2000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.200000000e+01


## Results

In [8]:
print(f'Minimum distance: {m.objVal} miles')

for v in m.getVars():
    if abs(v.X) > 0.00001:
        print(f'{v.VarName}: {v.x}')

Minimum distance: 32.0 miles
path[1,3]: 1.0
path[2,4]: 1.0
path[3,2]: 1.0
path[4,6]: 1.0
